# **Note on using the program:**
*   Для удобства пользования программа разбита на блоки, каждый из которых выполняет свой набор задач;
*   Не все блоки программы являются обязательными к запуску для корректной работы программы;
*   Часть блоков могут являться взаимоисключающими (выполнять одну и туже задачу разными способами) или не быть важными для непосредственного запуска при каждом использовании программы у таких блоков возле номера находятся пометки "(alternative)" и "(optional)" соответственно;
>**Для безопасного и правильного использования программы - просматривайте описания для блоков перед их запуском.**




**block 1:**
Подключение необходимых для работы программы библиотек

In [ ]:
!pip install deepface
!pip install onnxruntime
!pip install insightface
!pip install gdown

import gdown # Библиотека, что позволяет быстро получить доступ к данным хранимым на google grive
import insightface
import pandas as pd # Библиотека для работы с csv файлом
import numpy as np
from insightface.app import FaceAnalysis # Библиотека для работы с лицами
from deepface import DeepFace # Библиотека для работы с лицами
import zipfile # Дополнение для работы с архивом фотографий
import shutil # Дополнение для перемещения файлов
import json
import cv2 # Дополнение для работы с видео-файлом
import os # Дополнение для создания папок

**block 1.1 (optional):**
Дополнение для запуска из google kolab (позволяет видеть изображения - не обязательно)

In [ ]:
from google.colab.patches import cv2_imshow # нужно при запуске из гугл колаб функции imshow

#block 2 and block 3:

> **Для дальнейшего использования программы необходимо добавить ссылки на днные по описанным ниже правилам, данные исходного эксперимента не являются публичными, поэтому недоступны для тестового запуска.**

block 2 и block 3 являются взаимозаменяемыми и реализуют загрузку файлов (архив с фотографиями, информация о людях с фотографий, видеозаписи для поиска)в среду выполнения, разница в том, что:

*   block 2 выполняет задачу используя библиотеку gdown, преназначенную для загрузки файлов с гугл диска, но не способную загружать их из других источников;
*   Block 3 выполняет задачу используя urllib.request, что позволяет загружать данные из любых свободных источников в интернете, но могут возникать сложность при загрузки больших файлов (более 500 Мб).

> **В данном решении рекомендуется использовать block 2.**

**Примечание:** для использования файлов с гугл диска нужно изменить формат ссылки с
https://drive.google.com/file/d/THE_LINK_TO_YOUR_FILE/view?usp=drive_link
на
https://docs.google.com/uc?id=THE_LINK_TO_YOUR_FILE

**block 2 (alternative):**

In [ ]:
files_url = {
    "photo_staff.zip" : "" # Архив исходных фото
    ,"emb_staff_dict.json" : "" # (не обязательно) загрузка файла с эмбедингами пригодных изображений - создается в Block 6 и может использоваться повторно
    ,"names.csv" : "" # Информация с именами, для идентификации вида: название фото - ФИО человека на фото
    ,"video1.mp4": "" # Первая видео-запись
    ,"video2.mp4": "" # Вторая видео-запись
    ,"video3.mp4": "" # Третья видео-запись
}

for file in files_url:
  gdown.download(files_url[file], file, quiet=False)

original_zip = zipfile.ZipFile('/content/photo_staff.zip')
original_zip.extractall('/content')
original_zip.close()

**block 3 (alternative):**


In [ ]:
import urllib.request # Дополнение для загрузки необходимых данных

files_url = {
    "photo_staff.zip" : "" # Архив исходных фото
    ,"emb_staff_dict.json" : "" # (не обязательно) загрузка файла с эмбедингами пригодных изображений - создается в Block 6 и может использоваться повторно
    ,"names.csv" : "" # Информация с именами, для идентификации вида: название фото - ФИО человека на фото
    ,"video1.mp4": "" # Первая видео-запись
    ,"video2.mp4": "" # Вторая видео-запись
    ,"video3.mp4": "" # Третья видео-запись
}

for file in files_url:
     urllib.request.urlretrieve(files_url[file], file)

original_zip = zipfile.ZipFile('/content/photo_staff.zip')
original_zip.extractall('/content')
original_zip.close()


#block 4, block 5 and block 6:
В процессе разработки и оптимизации решения поставленной задачи, а именно, решая задачу верификации,  поскольку исходное множество лиц для поиска на видео велико - рассматривалась возможность сократить или видоизменить представление данного множества с целью снизить количесвто операций при верификации.

Наиболее очевидным и рациональным из подходов, применимых к данной задаче, с целью сокрщения количества проверок на одно фото был вариант разбиения исходного списка фотографий на несколько более малых, по половому признаку.

Собственно, в block 5 and block 6 представлены варианты программ, выполняющих данную задачу.
*   block 5 выполняет задачу используя библиотеку deepface;
*   Block 6 выполняет задачу используя insightface.

Оценивая результаты, полученные посредством выполнения данных программ, был выведен ряд наблюдений:
* Таковым является наличие ошибок, причем различных, которыми могут быть как ненахождение лица на фото, так и неверная трактовка пола, что особенно заметно при анализе лиц старшей возрастной группы;
* Дополнением к этому является наличие некачественных данных загруженных самими пользователями: встречаются фотографии, на которых нет лиц вовсе или фото нескольких человек, что с точки зрения решаемой задачи могло бы помешать идентификации этих людей. В очень редких случаях (1) фото оказывается нечитаемым вовсе;
* Исходя из замеченных выше деталей приходим к выводу о низкой эффективности применения данного подхода, поскольку при использовании любой из библиотек точность прогноза не превышает значение 80%, то есть используя это звено мы добавляем дополнительную не всегда работающую точно проверку, тем самым приводя к росты издержек, который может выражаться в необходимости дополнительных циклов верификации или потере потенциальных лиц при поиске.

Однако результаты проделанной работы могут быть использованы в других целях:
* В рамках разработки программы были получены возможность ознакомления и оценки качества используемых в ней библиотек. Решение на основе билиотеки deepface может похвастаться простатой внедрения, при работе с ним вы рассматриваете всего три случая: определен мужской пол, определен женский пол, произошла ошибка (ValueError). Что хорошо для новичка в данной сфере с одной стороны, но является не лучшим результатом для более квалифицированного специалиста, так как скрывает все возможные причины неполучения результата за одной ошибкой и не дает возможность увидеть ее причины глубже, что может быть критично, ведь неопознынный набор превысил 10% фотографий.
* Библиотека insightface в этоже время повышает уровень требований к ее пользвателю, но и дает больше возможностей, первым, что хочется отметить это лучшую способноть к распознаванию, количество неопознанных фото снизилось до 2%, вторым важным плюсом является возможность в отлавливании причин по которым фото оказалось неподходящим, так после работы программы block 6 мы получили 13 неопознанных фото, из которых одно низкого качества, на пяти фотографиях присутствует несколько человек и на 7 фото нет лиц. То есть используя данную библиотеку мы можем приоткрыть завесу черного ящика и отследить причину по которой фото "бракуется". Однако, как и говорилось выше, это накладывает более высокую ответственность на программиста, ведь, если не проявлять должной бдительности при отлове фото, 12 из 13 неопознанных можно было бы пропустить, что могло бы снизить качество разработанного решения и добавить в него неверифицируемых людей, при этом нигде явно об этом не сообщая.
* Разработанная в block 6 программа может использоваться в дальнейшем, если не для разбора пола, что она делает с вероятностью меньшей 80%, то для поиска и удаления неподходящих фото - с целью оповестить потенциальных пользователей о необходимости замены их фото на новые.

> Block 5 и Block 6 являются не обязательными для запуска и использования программы, но позволяют ознакомиться с результатами проделанных экспериментов. Они работоспособны и могут быть запущенных в общем потоке программы, однако их работа может занять более 5 минут времени.

> **Если вас интересуют исключительно результаты работы данных блоков - можно воспользоваться block 4, который загрузит архивы содержащие их в общую папку, откуда вы сможете их сразу загрузить без ожидания их выполнения.**

> **В дальнейшей работе программы не будут использоваться те 13 фотографий, которые были определены как не качественные, подобная отчистка хоть и незначильно но позвоялет приблизиться к цели сокращения затрат времени на верификацию человека на видео.**

**block 4 (optional):** Загрузка результатов, которые можно получить выполнением block 5 и block 6.:

>**Данный блок недоступен для выполнения из-за требований правообладателя, данные испольхуемые в нем изъяты из публичного доступа. Вы можете указать ссылки на свои данные для его использования.**

In [ ]:
"""
files_url = {
    "gender_staff_deepface.zip" : "" # Архив результатов для deepface
    ,"gender_staff_insightface.zip" : "" # Архив результатов для insightface
    ,"photo_staff_clean.zip" : "" # Архив отчищенных фото
}

for file in files_url:
  gdown.download(files_url[file], file, quiet=False)

# Раскомментировать код внизу, если помимо загрузки нужно распаковать архивы (Нужно для работы block 7)
"""
"""
with zipfile.ZipFile('/content/gender_staff_deepface.zip', 'r') as gender1_zip:
  gender1_zip.extractall('/')
gender1_zip.close()

with zipfile.ZipFile('/content/gender_staff_insightface.zip', 'r') as gender2_zip:
  gender2_zip.extractall('/')
gender2_zip.close()

with zipfile.ZipFile('/content/photo_staff_clean.zip', 'r') as gender3_zip:
  gender3_zip.extractall('/')
gender3_zip.close()
"""

**block 5 (optional):** создание папок и сортировка изображений на три группы: man, woman and unrecognized, с использованием библиотеки deepface. Сворачивание результатов работы в zip-архив для извлечения.

In [ ]:
os.makedirs('man_deepface', exist_ok=True)
os.makedirs('woman_deepface', exist_ok=True)
os.makedirs('unrecognized_deepface', exist_ok=True)

dataset_path = os.listdir('photo_staff')
zip_deepface = zipfile.ZipFile('/content/gender_staff_deepface.zip', 'w', zipfile.ZIP_DEFLATED)

def gender_analyze(img_name):
    try:
      result = DeepFace.analyze('/content/photo_staff/'+img_name, actions=['gender'])
      if result[0]['gender']['Man'] > result[0]['gender']['Woman']:
          shutil.copy('/content/photo_staff/'+img_name, '/content/man_deepface')
          zip_deepface.write('/content/man_deepface/'+img_name)
      else:
          shutil.copy('/content/photo_staff/'+img_name, '/content/woman_deepface')
          zip_deepface.write('/content/woman_deepface/'+img_name)
    except ValueError:
      shutil.copy('/content/photo_staff/'+img_name, '/content/unrecognized_deepface')
      zip_deepface.write('/content/unrecognized_deepface/'+img_name)
      print(f"Фото {img_name} не подлежит распознаванию")

for image_name in dataset_path:
    #print(f"Начали анализ фото {image_name}")
    #print(f"Начали анализ фото {'/content/photo_staff/'+image_name}")
    gender_analyze(image_name)
zip_deepface.close()


Фото 327_zulip.jpeg не подлежит распознаванию
24-04-06 11:01:01 - gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:10<00:00, 50.3MB/s]


Фото 235_miem.jpeg не подлежит распознаванию
Фото 83_miem.jpeg не подлежит распознаванию
Фото 421_miem.jpeg не подлежит распознаванию
Фото 345_miem.jpeg не подлежит распознаванию
Фото 431_miem.jpeg не подлежит распознаванию
Фото 56_miem.jpeg не подлежит распознаванию
Фото 488_zulip.jpeg не подлежит распознаванию
Фото 388_miem.jpeg не подлежит распознаванию
Фото 132_miem.jpeg не подлежит распознаванию
Фото 19_zulip.jpeg не подлежит распознаванию
Фото 491_zulip.jpeg не подлежит распознаванию
Фото 425_miem.jpeg не подлежит распознаванию
Фото 134_zulip.jpeg не подлежит распознаванию
Фото 360_miem.jpeg не подлежит распознаванию
Фото 74_zulip.jpeg не подлежит распознаванию
Фото 145_miem.jpeg не подлежит распознаванию
Фото 137_miem.jpeg не подлежит распознаванию
Фото 368_miem.jpeg не подлежит распознаванию
Фото 297_miem.jpeg не подлежит распознаванию
Фото 180_miem.jpeg не подлежит распознаванию
Фото 294_miem.jpeg не подлежит распознаванию
Фото 183_miem.jpeg не подлежит распознаванию
Фото 428_

**block 6 (optional):** создание папок и сортировка изображений на три группы: man, woman and unrecognized, с использованием библиотеки insightface. Сворачивание результатов работы в zip-архив для извлечения. Создание Json файла с эмбедингами фотографий признанных пригодными для работы.

In [ ]:
os.makedirs('man_insightface', exist_ok=True)
os.makedirs('woman_insightface', exist_ok=True)
os.makedirs('unrecognized_insightface', exist_ok=True)

os.makedirs('photo_staff_clean', exist_ok=True)
emb_staff_dict = {}

dataset_path = os.listdir('photo_staff')
zip_insightface = zipfile.ZipFile('/content/gender_staff_insightface.zip', 'w', zipfile.ZIP_DEFLATED)
zip_clean = zipfile.ZipFile('/content/gender_staff_clean.zip', 'w', zipfile.ZIP_DEFLATED)

model_gender = FaceAnalysis(name="buffalo_l")
model_gender.prepare(ctx_id=0, det_size=(256,256))

def gender_analyze(img_name):
    try:
      fail = 0
      result = model_gender.get(cv2.imread('/content/photo_staff/'+img_name))
      if result == []:
        fail += 1
        shutil.copy('/content/photo_staff/'+img_name, '/content/unrecognized_insightface')
        zip_insightface.write('/content/unrecognized_insightface/'+img_name)
        zip_clean.write('/content/unrecognized_insightface/'+img_name)
        print(f"Фото {img_name} не подлежит распознаванию - не найдено лицо")
        return 0
      if len(result) != 1 and fail == 0:
        fail += 1
        shutil.copy('/content/photo_staff/'+img_name, '/content/unrecognized_insightface')
        zip_insightface.write('/content/unrecognized_insightface/'+img_name)
        zip_clean.write('/content/unrecognized_insightface/'+img_name)
        print(f"Фото {img_name} не подлежит распознаванию - найдено более одного лица")
        return 3
      #print(result)
      #print("")
      if fail == 0:
        for face in result:
          result_gender = face.gender
          emb_staff_dict[img_name] = (face.embedding).tolist()
          #print(f"Фото {img_name} номер пола {result_gender}")
          if result_gender == 1:
              shutil.copy('/content/photo_staff/'+img_name, '/content/man_insightface')
              shutil.copy('/content/photo_staff/'+img_name, '/content/photo_staff_clean')
              zip_insightface.write('/content/man_insightface/'+img_name)
              zip_clean.write('/content/photo_staff_clean/'+img_name)
          else:
              shutil.copy('/content/photo_staff/'+img_name, '/content/woman_insightface')
              shutil.copy('/content/photo_staff/'+img_name, '/content/photo_staff_clean')
              zip_insightface.write('/content/woman_insightface/'+img_name)
              zip_clean.write('/content/photo_staff_clean/'+img_name)
        return 1
    except AttributeError:
      shutil.copy('/content/photo_staff/'+img_name, '/content/unrecognized_insightface')
      zip_insightface.write('/content/unrecognized_insightface/'+img_name)
      zip_clean.write('/content/unrecognized_insightface/'+img_name)
      print(f"Фото {img_name} не подлежит распознаванию - качество фото (AttError)")
      return 2

for image_name in dataset_path:
    #print(f"Начали анализ фото {image_name}")
    #print(f"Начали анализ фото {'/content/photo_staff/'+image_name}")
    def_state = gender_analyze(image_name)
    if def_state != 1:
      print(def_state)
with open('emb_staff_dict.json', 'w') as f:
    json.dump(emb_staff_dict, f)
zip_insightface.close()
zip_clean.close()

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:04<00:00, 60573.35KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (256, 256)
Фото 327_zul

**block 7 (optional):** Подсчет количества изображений в разных папках.
> В данном блоке можно ознакомиться с результатами разных библиотек на представленном списке фотографий. Данные показатели могут использоваться для оценки эффективности решений или определения иных статистических показателей.

> **Данный блок не будет работать, если не запустить block 4 или совместно block 5 и block 6.**

In [ ]:
folder_path_list = ['/content/woman', '/content/man', '/content/unrecognized']
file_add_deepface = 0 # Инициализируем переменные для хранения сумм файлов
file_add_insightface = 0
file_add_clean = 0
file_count_clean = 0
file_unrecognized_insightface = 0

file_count_base = 0 # Инициализируем счетчик файлов исходной папки

for root, dirs, files in os.walk('/content/photo_staff'):
  file_count_base += len(files)
print(f"Количество файлов папке {'/content/photo_staff'}: {file_count_base}\n")

for folder_path in folder_path_list:
  file_count_deepface = 0 # Инициализируем счетчики файлов
  file_count_insightface = 0

  for root, dirs, files in os.walk(folder_path+'_deepface'): # Осуществляем обход всех элементов папки
      file_count_deepface += len(files) # Для каждого файла увеличиваем счетчик

  for root, dirs, files in os.walk(folder_path+'_insightface'):
      file_count_insightface += len(files)

  print(f"Количество файлов папке {folder_path+'_deepface'}: {file_count_deepface}")
  print(f"Количество файлов папке {folder_path+'_insightface'}: {file_count_insightface}\n")

  file_unrecognized_insightface = file_count_insightface

  file_add_deepface += file_count_deepface
  file_add_insightface += file_count_insightface

for root, dirs, files in os.walk("/content/photo_staff_clean"):
    file_count_clean += len(files)
print(f"Количество файлов в папке /content/photo_staff_clean: {file_count_clean}\n")

print(f"Количество файлов в clean архиве: {file_count_clean + file_unrecognized_insightface}")
print(f"Количество файлов в deepface папках: {file_add_deepface}")
print(f"Количество файлов в insightface папках: {file_add_insightface}")



Количество файлов папке /content/photo_staff: 464

Количество файлов папке /content/woman_deepface: 74
Количество файлов папке /content/woman_insightface: 123

Количество файлов папке /content/man_deepface: 332
Количество файлов папке /content/man_insightface: 328

Количество файлов папке /content/unrecognized_deepface: 58
Количество файлов папке /content/unrecognized_insightface: 13

Количество файлов в папке /content/photo_staff_clean: 451

Количество файлов в clean архиве: 464
Количество файлов в deepface папках: 464
Количество файлов в insightface папках: 464


**block 8:**
> В данном блоке задаем основные зависящие от нас параметры обработки видео: frame_start, step and videos.
*   frame_start - стартовый фрейм с которого начинается осмотр видео (по умолчанию поставлен 0);
*   step - шаг разбиения, количество фреймов из видео, которые мы будем анализировать (анализ каждого кадра видео потребует слишком многого процессорного времени, от того было решено - выбрать некоторое количество
кадров, через равные по размеру промежутки, тем самым охватив все видео)
*   videos - список, в котором нужно указать наименования загруженных видео-файлов, которые подлежат анализу. По умолчанию указаны загруженные выше видео;
* Происходит загрузка Json-файла с эмбедингами лиц преподавателей, которых собираемся искать на видео;
* found_photo_dict - словарь, для хранения найденных лиц в видео в форме {фотография : количество обнаружений}.





In [ ]:
emb_staff_dict = {}

with open('emb_staff_dict.json', 'r') as f:
    emb_staff_dict = json.load(f)
for key, value in emb_staff_dict.items():
  emb_staff_dict[key] = np.array(value)

videos = ["video1.mp4", "video2.mp4", "video3.mp4"] # Список загруженных для анализа видео
frame_start = 0 # стартовый фрейм
step = 400 # Количетсво кадров видео, которые будут осмотрены (из-за особенностей реализации, почти всегда будет на 1 кадр меньше)
found_photo_dict = {}

def AddPhotoToDict(key): # Функция добавляет в словарь информацию о найденных лицах
  if found_photo_dict.get(key) is not None:
    found_photo_dict[key] += 1
  else:
    found_photo_dict[key] = 1

**block 9:** Анализ видеопотока, проведение верефикации и вывод рехультатов.

In [ ]:
model = FaceAnalysis(name="buffalo_l")
model.prepare(ctx_id=0, det_size=(256,256))

for video_now in videos:

  frame_now = frame_start
  cap = cv2.VideoCapture('/content/'+video_now)
  size_cap = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  step_cap = int(size_cap / step) # размер шага (на сколько делим число кадров, столько кадров и возьмем для анализа)
  print(f"\nНачалась обработка видео: {video_now}")
  print(f"Всего кадров в {video_now}:", size_cap)

  while(size_cap > frame_now):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_now)
    ret, frame = cap.read()
    if ret:
      #cv2.imshow('Frame', frame) # не работает в гугл колаб, ниже реализована замена специально под него
      #print("Это кадр №", frame_now)
      #cv2_imshow(frame) # выводит на экран выбранный в данный момент кадр видео
      #cv2.waitKey(0)    # Показывает этот кадр пока не нажата любая клавиша (в гугл колаб - просто показывает кадр)
      #print('\n')
      faces = model.get(frame)
      if (faces != []):
        for face in faces:
          face_emb = face.embedding
          for key in emb_staff_dict:
            #print(face_emb)
            distance = np.linalg.norm(emb_staff_dict[key] - face_emb)
            #print(key, distance)
            threshold = 20 # можно менять, в случае, если находится слишком много совпадений
            if distance < threshold:
              AddPhotoToDict(key)
              #print(face_emb)
              #cv2_imshow(frame)
              #bbox = face.bbox
              #print(face.bbox)
              #cv2_imshow(frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])])
    else:
      print("Не удалось прочитать кадр")
    frame_now = frame_now + step_cap
  if len(found_photo_dict) != 0:
    first_true_detect = 0
    for key in found_photo_dict:
      if found_photo_dict[key] > 1 and first_true_detect == 0:
        data = pd.read_csv('names.csv')
        first_true_detect = 1
        print(f"На видео {video_now} потенциально могут присутствовать:")
        for index, row in data.iterrows():
          if row[4] == key:
            print(f"человек с фото {row[4]}, а именно {row[1]}")
      elif found_photo_dict[key] > 1:
        for index, row in data.iterrows():
          if row[4] == key:
            print(f"человек с фото {row[4]}, а именно {row[1]}")
    if (first_true_detect == 0):
      print(f"На видео {video_now} никого не удалось обнаружить.")
  else:
    print(f"На видео {video_now} никого не удалось обнаружить.")
  print(found_photo_dict)
  found_photo_dict = {}

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (256, 256)

Началась об

# **Results:**
>**Результаты проделанной работы не позволяют однозначно говорить о потенциале применения бибилоитек deepface и insightface в задаче верификации челвоека на видео. Поскольку основной вопрос справедливости оценивания замыкается на качестве исходных данных, а именно, на качестве изображений верифицируемых людей.**

Общией рекомендацией для осуществления дальнейших работ может являться:

* Повышение качества используемых материалов;
* Контроль за соответствием фотографии разыскиваемого пользователя его реальному виду;
* Повышение требований к разрешению фотографий и ограничение использования фотографий в оттенках серого;
* Повышение качества средств сбора видеопотока.


